In [1]:
!pip install evaluate --quiet

In [2]:
!pip install gdown --quiet

In [3]:
from datasets import load_dataset
from nltk import sent_tokenize, word_tokenize
import random
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
import os
import torch
import evaluate
import numpy as np

In [4]:
data = load_dataset('onestop_english')

Generating train split:   0%|          | 0/567 [00:00<?, ? examples/s]

Dataset onestop_english downloaded and prepared to /root/.cache/huggingface/datasets/onestop_english/default/1.1.0/6b19eec5680862ad1cf1990e98b06a98d1fa4c85f3585dc4dfab93f52b89d9cf. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 567
    })
})

In [6]:
sentences_hard = []
sentences_simple = []
for i in range(len(data['train'])):
    if data['train'][i]['label'] == 2:
        for sentence in sent_tokenize(data['train'][i]['text']):
            sentences_hard.append(' '.join(word_tokenize(sentence.lower())))
    elif data['train'][i]['label'] == 0:
        for sentence in sent_tokenize(data['train'][i]['text']):
            sentences_simple.append(' '.join(word_tokenize(sentence.lower())))

In [7]:
!gdown https://drive.google.com/uc?id=13BC8BDL2MkksUtqsQ1clPxcPvKWgKlO3 --quiet
!gdown https://drive.google.com/uc?id=1zUMoJk3Jkf9wh40tRG1lSI07RDMp_l8n --quiet
!gdown https://drive.google.com/uc?id=19uhGuMkTeV_nveGQ2rbtbitQMI0b6Xsp --quiet
!gdown https://drive.google.com/uc?id=1bmw6wGVt5h52w3WEVVndrucEDBwws66l --quiet

In [8]:
def read_turkers(path):
    with open(path, 'r') as f:
        sentences = f.read().splitlines()
    return sentences

In [9]:
turkers_hard_one = read_turkers('/kaggle/working/test.8turkers.tok.norm.txt')
turkers_hard_two = read_turkers('/kaggle/working/tune.8turkers.tok.norm.txt')
turkers_simple_one = read_turkers('/kaggle/working/test.8turkers.tok.simp.txt')
turkers_simple_two = read_turkers('/kaggle/working/tune.8turkers.tok.simp.txt')

In [10]:
sentences_hard += turkers_hard_one
sentences_hard += turkers_hard_two
sentences_simple += turkers_simple_one
sentences_simple += turkers_simple_two

In [11]:
random.shuffle(sentences_hard)
random.shuffle(sentences_simple)

In [12]:
sentences_hard_train = sentences_hard[1001:len(sentences_hard)]
sentences_hard_test = sentences_hard[0:1000]

sentences_simple_train = sentences_simple[1001:len(sentences_simple)]
sentences_simple_test = sentences_simple[0:1000]

In [13]:
def collect_dataset(sentences_hard, sentences_simple):
    dataset = []
    for sentence_hard in sentences_hard:
        dataset.append({'text': sentence_hard, 'label': 1})
    for sentence_simple in sentences_simple:
        dataset.append({'text': sentence_simple, 'label': 0})
    random.shuffle(dataset)
    return dataset

In [14]:
train_dataset = pd.DataFrame.from_dict(collect_dataset(sentences_hard_train, sentences_simple_train))
test_dataset = pd.DataFrame.from_dict(collect_dataset(sentences_hard_test, sentences_simple_test))

In [15]:
datasets_train_test = DatasetDict({
    "train": Dataset.from_pandas(train_dataset),
    "test": Dataset.from_pandas(test_dataset)
    })

In [16]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = torch.device('cuda:0')
device = torch.device('cpu')

In [17]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base')
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [18]:
model.to(device);

In [19]:
def tokenization(text):
    return tokenizer(text["text"], padding='max_length', truncation=True)

In [20]:
datasets_train_test = datasets_train_test.map(tokenization, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [21]:
datasets_train_test

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 14827
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [22]:
datasets_train_test = datasets_train_test.class_encode_column('label')
datasets_train_test.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Stringifying the column:   0%|          | 0/15 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/15 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

Stringifying the column:   0%|          | 0/2 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/2 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [23]:
clf_metrics = evaluate.combine(["accuracy"])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return clf_metrics.compute(predictions=predictions, references=labels)

In [24]:
training_args = TrainingArguments(
    output_dir='results',
    save_total_limit=2,
    load_best_model_at_end=True,
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    warmup_steps=10
)

In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets_train_test['train'],
    eval_dataset=datasets_train_test['test'],
    compute_metrics = compute_metrics
)

In [26]:
os.environ["WANDB_DISABLED"] = "true"

In [27]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14827
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1854
  Number of trainable parameters = 124647170
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy
1,0.589300,0.530447,0.743500
2,0.423700,0.595662,0.735500


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 32
Saving model checkpoint to results/checkpoint-927
Configuration saved in results/checkpoint-927/config.json
Model weights saved in results/checkpoint-927/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 32
Saving model checkpoint to results/checkpoint-1854
Configuration saved in results/checkpoint-1854/config.json
Model weights saved in results/checkpoint-1854/p

TrainOutput(global_step=1854, training_loss=0.49523701096815587, metrics={'train_runtime': 2761.809, 'train_samples_per_second': 10.737, 'train_steps_per_second': 0.671, 'total_flos': 7802295235645440.0, 'train_loss': 0.49523701096815587, 'epoch': 2.0})

In [29]:
!tar -czf results.tar.gz /kaggle/working/results

tar: Removing leading `/' from member names
^C


In [31]:
os.chdir(r'/kaggle/working')

In [32]:
from IPython.display import FileLink
FileLink(r'results.tar.gz')

/kaggle/working/results.tar.gz